In [5]:
import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)
    
catchVar = data_dict.pop('TOTAL', None)
 
# data = featureFormat(data_dict, features_list, sort_keys = True)

1. Summarize for us the goal of this project and how machine learning is useful in trying to accomplish it. As part of your answer, give some background on the dataset and how it can be used to answer the project question. Were there any outliers in the data when you got it, and how did you handle those?  [relevant rubric items: “data exploration”, “outlier investigation”]

The goal of the project is to find additional persons of interest in the enron dataset based on an already identified list of persons of interest. Machine learning is a useful tool because we have a semi-large set of multivariate data to look through which would be very difficult to do by hand or by an eyeball test. We also have emails from many people in the company which we can analyze. 

We are essentially looking for data that identifies someone as a person of interest. Personally, I think it is most interesting if we can understand what makes a person a person of interest, but that may not be possible give the data we have. One thing that I would like to understand is how money plays into who is a person of interest. It would make sense for poi's to have a large monetary incentive to make the company look like it is doing better than it actually is in the short term.  

An outlier in the dataset it 'TOTAL', which is the total from each category. This has been removed. It was easy to identify because a, the name is a give away but also because it was the max value in 7 features and a max value by a large margin. After removing TOTAL, Kenneth Lay is the heighest in 6 categories but as the head of the company this makes sense. A counter point is he may not help determine other persons of interest because his earnings for out weigh other employees. This may be true of other top earner pois. BELFER ROBERT has the most categories with the minimum. Not sure if this is relevant yet.

In [8]:
len(data_dict.keys()) #total number of people to look at

145

In [16]:
from helpers import orig_pois
from helpers import reduceToFeature
pois = orig_pois()
print len(pois) # number of original pois
print len([0 for poi in reduceToFeature(data_dict, 'poi') if poi]) # number of pois in dataset

35
18


In [14]:
data_dict[data_dict.keys()[0]].keys() #keys

['salary',
 'to_messages',
 'deferral_payments',
 'total_payments',
 'exercised_stock_options',
 'bonus',
 'restricted_stock',
 'shared_receipt_with_poi',
 'restricted_stock_deferred',
 'total_stock_value',
 'expenses',
 'loan_advances',
 'from_messages',
 'other',
 'from_this_person_to_poi',
 'poi',
 'director_fees',
 'deferred_income',
 'long_term_incentive',
 'email_address',
 'from_poi_to_this_person']

In [7]:
from helpers import financial_features
from helpers import numerical_email_features

features_list = ['poi'] + financial_features

data = featureFormat(data_dict, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)



[  0.00000000e+00   2.01955000e+05   2.86971700e+06   4.48444200e+06
   1.72954100e+06   4.17500000e+06   1.26027000e+05  -1.26027000e+05
   1.72954100e+06   1.38680000e+04   0.00000000e+00   1.52000000e+02
   0.00000000e+00  -3.08105500e+06   3.04805000e+05]


Question: What features did you end up using in your POI identifier, and what selection process did you use to pick them? Did you have to do any scaling? Why or why not? As part of the assignment, you should attempt to engineer your own feature that does not come ready-made in the dataset -- explain what feature you tried to make, and the rationale behind it. (You do not necessarily have to use it in the final analysis, only engineer and test it.) In your feature selection step, if you used an algorithm like a decision tree, please also give the feature importances of the features that you use, and if you used an automated feature selection function like SelectKBest, please report the feature scores and reasons for your choice of parameter values.  [relevant rubric items: “create new features”, “properly scale features”, “intelligently select feature”]

Answer: I ended up picking a 9 component PCA. I first looked at financial and email features seperetly and looked at recursive feature selection using an AdaBoostClassifier. For financial data, 8 features very clearly were not useful for predicting persons of interest: 
    - deferral_payments
    - total_payments
    - restricted_stock
    - restricted_stock_deferred
    - total_stock_value
    - loan_advances
    - director_fees
    - long_term_incentive
    
The other 6 financial features:
    - salary
    - exercised_stock_options
    - bonus
    - expenses
    - other
    - deferred_income

Do seem to matter. I am skeptical if all of these features really matter but will go with it for now. 
    
For the 4 email features I had a hard time getting consistant results while changing the cv number. This made me think there are a few results that are pulling the results in one direction. Overall it did seem like all 4 features mattered. 

Before doing PCA I did MinMaxScaling on all the features. This seemed like it made sense because it would make it easier to compare email counts and $. 

Unfortunetly 6 + 4 != 9. I ran PCA for every possible number of components, 1-14 and looked at the AdaBoostClassifier score from resulting feature set. I looks like 9 is the best result. There is a smaller peak at 3 which I want to compare the end results of 9 to. 

It may also be that I need to construct my own features based on relationships between  peices of financial data. 

Question: What algorithm did you end up using? What other one(s) did you try? How did model performance differ between algorithms?  [relevant rubric item: “pick an algorithm”]

Answer: I chose AdaBoostClassification becasue I like this algorithm haha. It is fast and generally feels like it comes up with a good (not ridiculous but decently accurate) solution. I tried SVMs for some of the testing but it was running so slowly that I moved away from it. I also used DecisionTree's for some testing but wasn't happy with the results I was getting. 

Question: What does it mean to tune the parameters of an algorithm, and what can happen if you don’t do this well?  How did you tune the parameters of your particular algorithm? (Some algorithms do not have parameters that you need to tune -- if this is the case for the one you picked, identify and briefly explain how you would have done it for the model that was not your final choice or a different model that does utilize parameter tuning, e.g. a decision tree classifier).  [relevant rubric item: “tune the algorithm”]

Answer: If you don't tune your algorithm well you can have slower performace and over or under fitting. Tuning is a great way to figure out what might be best for your set of data. A great way to do tuning is to do a GridSearchCV with a wide array of parameters. This is great because it helps you hone in on a good set of params by running one program. AdaBoostClassifier does nto really have any params to tune so I didn't do any tuning. 